In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName("Python Spark SQL basic example") \
  .config("spark.some.config.option", "some-value") \
  .getOrCreate()

## 8. Join
- 선택한 조인 타입에 따라 조인의 성능에 영향을 미치
- 조인할 때 익스큐터에서 실행 파일 간의 데이터 셔플링을 필요로 하기 때문에 다른 조인과 조인순서를 고려해야함

- 스파크의 조인 타입
  - inner join, outer join, left outer join, right outer join
  - left semi join : 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 있는 경우에는 키가 일치하는 왼쪽 데이터셋만 유지
  - left anti join : 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 없는 경우에는 키가 일치하지 않는 왼쪽 데이터 셋만 유지
  - natural join : 두 데이터셋에서 동일한 이름을 가진 컬럼을 암시적으로 결합하는 조인을 수행
  - cross join : 왼쪽 데이터셋의 모든 로우와 오른쪽 데이터 셋의 모든 로우를 조합
  
- 주요 조인
![img](https://miro.medium.com/max/2492/0*jfglHEcbPtwv50J1.png)
![join_visualize](https://i0.wp.com/www.powerbi-pro.com/wp-content/uploads/2018/05/051118_1848_PowerBIsech1.png?fit=727%2C437&ssl=1)

### 8.1 조인 타입

In [5]:
# 예제에서 사용할 데이터 셋
person = spark.createDataFrame([
  (0, "Bill Chamber", 0, [100]),
  (1, "Matel Zaharia", 1, [500, 250, 100]),
  (2, "Michael Arnbrust", 1, [250, 100])])\
.toDF("id", "name", "graduate_program", "spark_status")

graduateProgram = spark.createDataFrame([
  (0, "Masters", "School of Information", "UC Berkeley"),
  (2, "Masters", "EECS", "UC Berkeley"),
  (1, "Ph. D", "EECS", "UC Berkeley")])\
.toDF("id", "degree", "department", "school")

sparkStatus = spark.createDataFrame([
  (500, "Vice President"),
  (250, "PMC Member"),
  (100, "Contributor")])\
.toDF("id", "status")

In [6]:
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

In [7]:
print("person")
display(person.limit(20))

id,name,graduate_program,spark_status
0,Bill Chamber,0,List(100)
1,Matel Zaharia,1,"List(500, 250, 100)"
2,Michael Arnbrust,1,"List(250, 100)"


In [8]:
print("graduateProgram")
display(graduateProgram.limit(20))

id,degree,department,school
0,Masters,School of Information,UC Berkeley
2,Masters,EECS,UC Berkeley
1,Ph. D,EECS,UC Berkeley


In [9]:
print("sparkStatus")
display(sparkStatus.limit(20))

id,status
500,Vice President
250,PMC Member
100,Contributor


### 8.3 내부 조인 inner join
- join type 을 명시하지 않았을 때 기본적으로 내부조인을 수행함
- 왼쪽 테이블과 오른쪽 테이블에서 동일한 컬럼을 가져야함
- 참으로 평가되는 로우만 결합
- 세 번째 파라미터로 조인 타입을 명확하게 지정할 수 있음
- 성능 팁 : 두 테이블의 키가 중복되거나 여러 복사본으로 있다면 성능이 저하됨. 조인이 여러 키를 최소화하기 위해 일종의 카테시안 조인으로 변환되어 버림

In [11]:
joinExpression = person["graduate_program"] == graduateProgram["id"]
display(person.join(graduateProgram, joinExpression).limit(10))

id,name,graduate_program,spark_status,id,degree,department,school
0,Bill Chamber,0,List(100),0,Masters,School of Information,UC Berkeley
1,Matel Zaharia,1,"List(500, 250, 100)",1,Ph. D,EECS,UC Berkeley
2,Michael Arnbrust,1,"List(250, 100)",1,Ph. D,EECS,UC Berkeley


In [12]:
joinExpression = person["graduate_program"] == graduateProgram['id']
display(person.join(graduateProgram, joinExpression, "inner").limit(10))

id,name,graduate_program,spark_status,id,degree,department,school
0,Bill Chamber,0,List(100),0,Masters,School of Information,UC Berkeley
1,Matel Zaharia,1,"List(500, 250, 100)",1,Ph. D,EECS,UC Berkeley
2,Michael Arnbrust,1,"List(250, 100)",1,Ph. D,EECS,UC Berkeley


### 8.4 외부 조인
- 왼쪽과 오른쪽의 모든 로우를 제공
- 왼쪽이나 오른쪽 DataFrame에 일치하는 로우가 없다면 해당 위치에 null을 삽입
- 성능 팁 : 공통 로우가 거의 없는 테이블에서 사용하면 결과값이 매우 커지고 성능이 저하함

In [14]:
joinExpression = person['graduate_program'] == graduateProgram['id']
display(person.join(graduateProgram, joinExpression, "outer").limit(10))

id,name,graduate_program,spark_status,id,degree,department,school
0,Bill Chamber,0,List(100),0,Masters,School of Information,UC Berkeley
1,Matel Zaharia,1,"List(500, 250, 100)",1,Ph. D,EECS,UC Berkeley
2,Michael Arnbrust,1,"List(250, 100)",1,Ph. D,EECS,UC Berkeley
null,null,null,null,2,Masters,EECS,UC Berkeley


### 8.5 왼쪽 외부 조인
- 왼쪽의 모든 로우와 오른쪽의 공통 로우(inner join)을 제공. 왼쪽에 일치하는 로우가 없으면 null을 채움

In [16]:
display(person.join(graduateProgram, joinExpression, "left_outer").limit(10))

id,name,graduate_program,spark_status,id,degree,department,school
0,Bill Chamber,0,List(100),0,Masters,School of Information,UC Berkeley
1,Matel Zaharia,1,"List(500, 250, 100)",1,Ph. D,EECS,UC Berkeley
2,Michael Arnbrust,1,"List(250, 100)",1,Ph. D,EECS,UC Berkeley


### 8.6 오른쪽 외부 조인
- 식의 순서만 바꾸면 왼쪽 외부 조인과 같은 역할
- 오른쪽의 모든 로우와 왼쪽의 모든 로우(inner join)을 제공. 오른쪽에 일치하는 로우가 없으면 null을 채움
  - 오른쪽을 기준으로 오늘쪽에 해당하는 값이 왼쪽에 없을 때 왼편에 값을 null로 채움

In [18]:
display(person.join(graduateProgram, joinExpression, "right_outer").limit(10))

id,name,graduate_program,spark_status,id,degree,department,school
0,Bill Chamber,0,List(100),0,Masters,School of Information,UC Berkeley
1,Matel Zaharia,1,"List(500, 250, 100)",1,Ph. D,EECS,UC Berkeley
2,Michael Arnbrust,1,"List(250, 100)",1,Ph. D,EECS,UC Berkeley
null,null,null,null,2,Masters,EECS,UC Berkeley


In [19]:
display(graduateProgram.join(person, joinExpression, "left_outer").limit(10))

id,degree,department,school,id,name,graduate_program,spark_status
0,Masters,School of Information,UC Berkeley,0,Bill Chamber,0,List(100)
1,Ph. D,EECS,UC Berkeley,1,Matel Zaharia,1,"List(500, 250, 100)"
1,Ph. D,EECS,UC Berkeley,2,Michael Arnbrust,1,"List(250, 100)"
2,Masters,EECS,UC Berkeley,null,null,null,null


### 8.7 왼쪽 세미 조인
- 오른쪽에 존재하는 것을 기반으로 왼쪽 로우만 제공
- 값이 존재하는지 확인하는 용도, 값이 있다면 왼쪽 DataFrame에 중복 키가 있더라도 로우는 결과에 포함
- 기존 조인 기능과는 달리 DataFrame의 필터 기능과 유사
- 성능 : 하나의 테이블만 확실히 고려되고, 다른 테이블은 조인 조건만 확인하기 때문에 성능이 매우 좋음

In [21]:
joinType = "left_semi"
display(graduateProgram.join(person, joinExpression, joinType).limit(10))

id,degree,department,school
0,Masters,School of Information,UC Berkeley
1,Ph. D,EECS,UC Berkeley


In [22]:
# 중복 키의 처리
gradProgram2 = graduateProgram.union(spark.createDataFrame([
  (0, "Masters", "Duplicated Row", "Duplicated School")
]))
gradProgram2.createOrReplaceTempView("gradProgram2")

display(gradProgram2.join(person, joinExpression, joinType).limit(10))

id,degree,department,school
0,Masters,School of Information,UC Berkeley
0,Masters,Duplicated Row,Duplicated School
1,Ph. D,EECS,UC Berkeley


### 8.8 왼쪽 안티 조인
- 왼쪽 세미 조인의 반대 개념. 즉, 오른쪽 DataFrame의 어떤 값도 포함하지 않음
- SQL의 NOT IN과 같은 스타일의 필터

In [24]:
display(graduateProgram)

id,degree,department,school
0,Masters,School of Information,UC Berkeley
2,Masters,EECS,UC Berkeley
1,Ph. D,EECS,UC Berkeley


In [25]:
joinType = "left_anti" # leftanti로 설정해도 됨
display(graduateProgram.join(person, joinExpression, joinType).limit(10))

id,degree,department,school
2,Masters,EECS,UC Berkeley


### 8.9 자연 조인
- 조인하려는 컬럼을 암시적으로 추정
- 암시적인 처리는 언제나 위험하므로 비추천
- Python join 함수는 이 기능을 지원하지 않음

```SQL
SELECT * FROM graduateProgram NATURAL JOIN person
```

In [27]:
display(spark.sql("""
  SELECT * FROM graduateProgram NATURAL JOIN person
""").limit(10))

id,degree,department,school,name,graduate_program,spark_status
0,Masters,School of Information,UC Berkeley,Bill Chamber,0,List(100)
1,Ph. D,EECS,UC Berkeley,Matel Zaharia,1,"List(500, 250, 100)"
2,Masters,EECS,UC Berkeley,Michael Arnbrust,1,"List(250, 100)"


### 8.10 교차 조인 cross join(카테시안 조인)
- 교차 조인은 조건절을 기술하지 않은 내부 조인을 의미
- 왼쪽의 모든 로우를 오른쪽의 모든 로우와 결합함(결과의 로우수 = 왼쪽 로우의 숫자 * 오른쪽 로우의 숫자)
- 성능 : 큰 데이터에서 사용할 경우 out-of-memory exception 발생. 가장 좋지 않은 성능을 가진 조인. 주의해서 사용하며 특정 사례에서만 사용해야 함

In [29]:
# 크로스 조인이지만 조건을 설정해야 하며, 조건에 부합된 결과를 출력하여 inner조인과 동일

joinType = "cross"
display(graduateProgram.join(person, on=joinExpression, how=joinType).limit(10))

id,degree,department,school,id,name,graduate_program,spark_status
0,Masters,School of Information,UC Berkeley,0,Bill Chamber,0,List(100)
1,Ph. D,EECS,UC Berkeley,1,Matel Zaharia,1,"List(500, 250, 100)"
1,Ph. D,EECS,UC Berkeley,2,Michael Arnbrust,1,"List(250, 100)"


In [30]:
# spark 2.1 이후
display(person.crossJoin(graduateProgram).limit(10))

id,name,graduate_program,spark_status,id,degree,department,school
0,Bill Chamber,0,List(100),0,Masters,School of Information,UC Berkeley
0,Bill Chamber,0,List(100),2,Masters,EECS,UC Berkeley
0,Bill Chamber,0,List(100),1,Ph. D,EECS,UC Berkeley
1,Matel Zaharia,1,"List(500, 250, 100)",0,Masters,School of Information,UC Berkeley
1,Matel Zaharia,1,"List(500, 250, 100)",2,Masters,EECS,UC Berkeley
1,Matel Zaharia,1,"List(500, 250, 100)",1,Ph. D,EECS,UC Berkeley
2,Michael Arnbrust,1,"List(250, 100)",0,Masters,School of Information,UC Berkeley
2,Michael Arnbrust,1,"List(250, 100)",2,Masters,EECS,UC Berkeley
2,Michael Arnbrust,1,"List(250, 100)",1,Ph. D,EECS,UC Berkeley


In [31]:
# spark 2.1 이후부터 : 조인에 조건을 부여하는 것을 잊으면 AnalysisException이 발생함

In [32]:
display(person.join(graduateProgram).limit(10))

org.apache.spark.sql.AnalysisException: Detected implicit cartesian product for INNER join between logical plans
Project [_1#16L AS id#24L, _2#17 AS name#25, _3#18L AS graduate_program#26L, _4#19 AS spark_status#27]
+- LogicalRDD [_1#16L, _2#17, _3#18L, _4#19], false
and
Project [_1#32L AS id#40L, _2#33 AS degree#41, _3#34 AS department#42, _4#35 AS school#43]
+- LogicalRDD [_1#32L, _2#33, _3#34, _4#35], false
Join condition is missing or trivial.
Either: use the CROSS JOIN syntax to allow cartesian products between these
relations, or: enable implicit cartesian products by setting the configuration
variable spark.sql.crossJoin.enabled=true;
	at org.apache.spark.sql.catalyst.optimizer.CheckCartesianProducts$$anonfun$apply$21.applyOrElse(Optimizer.scala:1413)
	at org.apache.spark.sql.catalyst.optimizer.CheckCartesianProducts$$anonfun$apply$21.applyOrElse(Optimizer.scala:1410)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$2.apply(TreeNode.scala:279)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$2.apply(TreeNode.scala:279)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDown(TreeNode.scala:278)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.org$apache$spark$sql$catalyst$plans$logical$AnalysisHelper$$super$transformDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.transformDown(AnalysisHelper.scala:149)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.transformDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.transformDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$3.apply(TreeNode.scala:284)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$3.apply(TreeNode.scala:284)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDown(TreeNode.scala:284)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.org$apache$spark$sql$catalyst$plans$logical$AnalysisHelper$$super$transformDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.transformDown(AnalysisHelper.scala:149)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.transformDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.transformDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$3.apply(TreeNode.scala:284)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$3.apply(TreeNode.scala:284)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDown(TreeNode.scala:284)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.org$apache$spark$sql$catalyst$plans$logical$AnalysisHelper$$super$transformDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.transformDown(AnalysisHelper.scala:149)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.transformDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.transformDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transform(TreeNode.scala:268)
	at org.apache.spark.sql.catalyst.optimizer.CheckCartesianProducts$.apply(Optimizer.scala:1410)
	at org.apache.spark.sql.catalyst.optimizer.CheckCartesianProducts$.apply(Optimizer.scala:1392)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.app

- AnalysisException을 무시하려면
  - ```spark.sql.crossJoin.enable=trueSpark``` 세션 빌더 객체에서 설정하거나
  - ```spark-shell에서 -conf spark.sql.crossJoin.enabled=true```로 변경해야 함
    - ```scala> spark.sparkContext.getConf.get("spark.sql.crossJoin.enable")```로 속성 설정 확인

**그 외 join에서 고려해야할 사항**
  - theta 조건을 충족하려면 join되는 [Theta Join - Tutorials point](https://www.tutorialspoint.com/dbms/database_joins.htm): key끼리 매치되도록 버켓을 만듦
  - 한 테이블의 한 열과 두번째 테이블의 여러 열과 매핑되는 One to Many Join : 한 열의 크기가 작으므로 일반적으로 메모리를 신경쓰지 않아도 됨. parquet을 사용하면 문제가 X
  - 하나의 테이블이 자신을 조인하는 self join: 데이터의 크기가 클 경우 out-of-memory 발생할 수 있음

### 8.11 조인 사용시의 문제점
#### 8.11.1 복합 데이터 타입의 조인
- 불리언을 반환하는 모든 표현식은 조인 표현식으로 간주할 수 있음

In [36]:
from pyspark.sql.functions import expr

display(person.withColumnRenamed("id", "personId") \
  .join(sparkStatus, expr("array_contains(spark_status, id)")).limit(10))

personId,name,graduate_program,spark_status,id,status
0,Bill Chamber,0,List(100),100,Contributor
1,Matel Zaharia,1,"List(500, 250, 100)",500,Vice President
1,Matel Zaharia,1,"List(500, 250, 100)",250,PMC Member
1,Matel Zaharia,1,"List(500, 250, 100)",100,Contributor
2,Michael Arnbrust,1,"List(250, 100)",250,PMC Member
2,Michael Arnbrust,1,"List(250, 100)",100,Contributor


#### 8.11.2 중복 컬럼명 처리

In [38]:
gradProgramDupe = graduateProgram.withColumnRenamed("id", "graduate_program")
joinExpr = gradProgramDupe['graduate_program'] == person['graduate_program']

In [39]:
# 조인을 수행했음에도 두 개의 gradudate_program 컬림이 존재
display(gradProgramDupe.join(person, joinExpr, "inner").limit(10))

graduate_program,degree,department,school,id,name,graduate_program,spark_status
0,Masters,School of Information,UC Berkeley,0,Bill Chamber,0,List(100)
1,Ph. D,EECS,UC Berkeley,1,Matel Zaharia,1,"List(500, 250, 100)"
1,Ph. D,EECS,UC Berkeley,2,Michael Arnbrust,1,"List(250, 100)"


In [40]:
# 중복 컬럼 조회 시 오류 발생
display(gradProgramDupe.join(person, joinExpr, "inner").select("graduate_program").limit(10))

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o1349.select.
: org.apache.spark.sql.AnalysisException: Reference 'graduate_program' is ambiguous, could be: graduate_program, graduate_program.;
	at org.apache.spark.sql.catalyst.expressions.package$AttributeSeq.resolve(package.scala:247)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveChildren(LogicalPlan.scala:103)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$42.apply(Analyzer.scala:937)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$42.apply(Analyzer.scala:939)
	at org.apache.spark.sql.catalyst.analysis.package$.withPosition(package.scala:53)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$.org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$resolve(Analyzer.scala:936)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$resolve$2.apply(Analyzer.scala:945)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$resolve$2.apply(Analyzer.scala:945)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$.org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$resolve(Analyzer.scala:945)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$apply$9$$anonfun$applyOrElse$36.apply(Analyzer.scala:1023)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$apply$9$$anonfun$applyOrElse$36.apply(Analyzer.scala:1023)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1$2.apply(QueryPlan.scala:122)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:104)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:122)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$4.apply(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.mapExpressions(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.

In [41]:
# 해결 방법 1 : 다른 조인 표현식 사용
## 중복된 두 컬럼 중 하나가 자동 제거
display(person.join(gradProgramDupe, "graduate_program").select("graduate_program").limit(10))

graduate_program
0
1
1


In [42]:
""" 해결 방법 2: 조인 후 컬럼 제거 """
display(gradProgramDupe.join(person, joinExpr).drop(gradProgramDupe["graduate_program"])\
    .select("graduate_program"))

graduate_program
0
1
1


In [43]:
# 해결 방법 3: 조인 전 컬러명 변경(가장 확실한 방법)
fixed_gradProgram = gradProgramDupe.withColumnRenamed("graduate_program", "grad_id")
display(fixed_gradProgram.join(person, fixed_gradProgram['grad_id'] == person['graduate_program']).limit(10))

grad_id,degree,department,school,id,name,graduate_program,spark_status
0,Masters,School of Information,UC Berkeley,0,Bill Chamber,0,List(100)
1,Ph. D,EECS,UC Berkeley,1,Matel Zaharia,1,"List(500, 250, 100)"
1,Ph. D,EECS,UC Berkeley,2,Michael Arnbrust,1,"List(250, 100)"


### 8.12 스파크의 조인 수행 방식
#### 8.12.1 네트워크 통신 전략
- **셔플조인** : 전체 노드 간 통신을 유발
- **브로드캐스트 조인** : 최초 테이블을 전체 노드로 복제 후 통신없이 진행
- 조인을 실행하려면 다층 익스큐터를 사용해 데이터 프레임의 파티션에서 동작
- 실제 작업과 성능은 조인의 타입과 조인되는 테이터셋의 특성에 따라 다름
- 크게 **Broadcast hash join, Shuffle Hash Join, Sort Merge Join**이 있음
- 세 가지 Join 방식을 그림으로 이해 
  - [ Spark SQL - 3 common joins (Broadcast hash join, Shuffle Hash join, Sort merge join) explained by Ram Ghadiyaram](https://www.linkedin.com/pulse/spark-sql-3-common-joins-explained-ram-ghadiyaram)

### 1. Broadcast Hash Join : SmallSize join Any Size
- 큰 데이터셋과 이보다 작은 데이터 셋 간의 조인은 큰 데이터셋의 파티션이 있는 모든 익스큐터에 작은 데이터셋이 브로드캐스트되어 수행
- 기본 값 : spark.sql.autoBroadcastJoinThreshold = 10mb
  - spark.sql.autoBroadcast.JoinThreshold 는 조인을 수행할 때 모든 작업자 노드에 브로드캐스트되는 테이블의 최대 크기를 구성
  - 참고
    - [Spark Doc - performance Tuning](https://spark.apache.org/docs/latest/sql-performance-tuning.html)
    - [Does spark.sql.autoBroadcastJoinThreshold work for joins using Dataset's join operator?](https://stackoverflow.com/questions/43984068/does-spark-sql-autobroadcastjointhreshold-work-for-joins-using-datasets-join-op)
    
- 수행
  - 작은 테이블은 드라이버에 모였다가 다시 모든 노드에 복사
  - 작은 테이블은 메모리에 올라가고
  - 큰 테이블은 스트림을 통해서 조인 수행
  - 브로드캐스트 힌트 ```person.join(broadcast(graduateProgram), Seq("id"))```
    - ```autoBroadcastJoinThreshold```에 관계없이 힌트가 있는 조인 측이 브로드캐스트됨.
    - 조인의 양쪽에 브로드캐스트 힌트가 있으면 실제 크기가 더 작은 쪽이 브로드 캐스트 됨
    - 힌트가 없고 테이블의 실제 물리적 추정값이 autoBroadcast.JoinThreshold 보다 작으면, 해당 테이블은 모든 실행기 노드로 브로드캐스트 됨.
    
- 성능
  - 한 쪽의 데이터가 하나의 machine 에 fit-in-memory 될 정도로 작으면 성능 좋음
    - table broadcast 는 네트워크를 많이 사용하므로, 브로드캐스트 된 테이블일 크면 때때로 out-of-memory 나 성능 저하가 발생할 수 있음
  - 셔플링이 없기 때문에, 브로드캐스트 되는 쪽이 데이터가 작으면 다른 알고리즘보다 빠름
  

- 브로드캐스트 지원
  - Full outer join은 지원하지 않음
  - left-outer join에서는 오른쪽 테이블만 브로드캐스트, right-outer join에서는 왼쪽 테이블만 브로드캐스트 가능

### Shuffle hash join : MiddleSize join LargeSize
- 두 테이블 모두 shuffle을 통해 노드에 분산되고
- 비교적 작은 테이블이 메모리 버퍼에 올라가고
- 큰 테이블은 스트림을 통해서 조인 수행
- 파티션이 전체 익스큐터로 분배
- 셔플은 비용이 많이 듦. 파티션과 셔플 배포가 최적으로 수행되는지 확인하기 위해 로직을 분석하는 것이 중요
  - 큰 데이터는 join이 필요한 부분만 filtering을 하거나
  - repartitioning을 고려해야함
  
- Map Reduce Fundamentals 유사
  - Map - 두 개의 서로 다른 Data frames/table
    - Output key를 join조건에서 필드로 사용
    - Shuffle - output key로 두 데이터 세트를 섞음
  - Reduce -join결과
  
![](https://camo.githubusercontent.com/e0be0071ea0207a39f4f2c049b449494d60044c0/68747470733a2f2f692e70696e696d672e636f6d2f6f726967696e616c732f34382f34312f38312f34383431383130646437616435303339376435363662386339626562373837352e6a7067)

**성능을 최적화하기 위해선**
- join할 키가 균등하게 distribute되어있거나
- parallelism위한 적절한 수의 키가 있을 때

**성능이 나쁜 경우 - 고르지 않은 sharding 및 제한된 parallelism**
- data skewness 처럼 하나의 단일 파티션이 다른 파티션에 비해 너무 많은 데이터를 가지고 있을 때
- 각 스테이트에서 50개 키만 셔플할 수 있음 -> 스파크 클러스터가 크면 고른 sharding과 parellelism으로 해결 못함

![](https://camo.githubusercontent.com/005808129f80059f9fe0f14212a95722a2418df9/68747470733a2f2f696d6167652e736c696465736861726563646e2e636f6d2f6f7074696d697a696e67737061726b73716c6a6f696e732d3137303230393136343633312f39352f6f7074696d697a696e672d6170616368652d737061726b2d73716c2d6a6f696e732d31312d3633382e6a70673f63623d31343836363538393137)

### Sort merge join : Large Size join LargeSize
- 일치하는 조인키를 sort할 수 있고, 브로드캐스트 조인, 셔플 해시 조인에 적합하지 않는 경우에 사용함
- shuffle hash join과 비교하면, 클러스터에서 데이터 이동(shuffle)을 최소화함
- 수행
  - 두 테이블 모두 셔플 및 정렬이 발생하고
  - 그나자 막은 쪽이 버퍼를 하고 큰 쪽이 스트리밍으로 조인을 수행
  - partition은 join 작업 전에 조인키 정렬
- 참고 : [SortMergeJoinExec Binary Physical Operator for Sort Merge Join](https://jaceklaskowski.gitbooks.io/mastering-spark-sql/spark-sql-SparkPlan-SortMergeJoinExec.html)

### BroadcastNetedLoopJoin
- 적용 : 조인 키가 지정되어 있지 않고 브로드 캐스트 힌트가 있거나 조인의 한쪽이 브로드캐스트 될 수 있고 spark.sql.autoBroadcastJoinThreshold 보다 작은 경우
- 브로드캐스트 된 데이터 세트가 크면 매우 느릴 수 있으며 OutOfMemoryException을 일으킬 수 있음

## 테이블 크기에 따른 조인 동작 방식
### 큰 테이블과 테이블 조인
- 전체 노드 간 통신이 발생하는 셔플 조인이 발생함

### 큰 테이블과 작은 테이블 조인
- 작은 DataFrame을 클러스터 전체 워커에 복제한 후 통신없이 진행
- 모든 단일 노드에서 개별적으로 조인이 수행되므로 CPU가 가장 큰 병목 구간이 됨
- broadcast 함수(힌트)를 통해 브로드캐스트 조인을 설정할 수 있지만 강제할 수는 없음(옵티마이저가 무시 가능)

In [50]:
from pyspark.sql.functions import broadcast

display(person.join(broadcast(gradProgramDupe), "graduate_program").select("graduate_program"))

graduate_program
0
1
1


### 작은 테이블과 작은 테이블 조인
- 스파크가 결정하도록 내버려두는 것이 가장 좋은 선택임

### Check
- 브로드캐스트 조인을 가능한 사용하고 조인 전에 관련없는 행을 조인 키로 필터링하여 불필요한 데이터 셔플링을 피하는 것이 좋음
  - 필요한 경우 **spark.sql.autoBroadcastJoinThrehold**를 적절하게 조정
- sort-merge join이 default이고 대부분의 시나리오에서 잘 수행됨
  - Shuffle Hash 조인이 Sort-Merge 조인보다 낫다고 확신이 있으면, Sort-Merge Join을 비활성화해서 shuffle hasg join이 수행되도록 하는 것이 좋음
    - build size가 stream size보다 작으면 Shuffle Hash 조인이 나음
- unique한 조인키가 없거나 조인키가 없는 조인은 수행비용이 비싸므로 최대한 피하는 것이 좋음

### 그 외 데이터 특성에 따라 고려해야할 것
- skewed data
  - [Spark summit rope 2017 - Working with Skewed Data: The Iterative Broadcast - Rob Keevil & Fokko Driesprong](https://www.youtube.com/watch?v=6zg7NTw-kTQ)
  - [The art of joining in Spark by Andrea Ialenti - Skew it! This is taking forever!](https://towardsdatascience.com/the-art-of-joining-in-spark-dcbd33d693c)
  - [Optimize the skew in Spark](https://unraveldata.com/common-failures-slowdowns-part-ii/)
  - [Data Skew and Garbage Collection to Improve Spark Performance](https://dataengi.com/2019/02/06/spark-data-skew-problem/)

## 파티셔닝 Partitioning
- 어떤 데이터를 어디에 저장할 것인지 제어할 수 있는 기능
- RDD는 데이터 파티션으로 구성되고 모든 연산은 RDD의 데이터 파티션에서 수행됨
- 파티션 개수는 RDD 트랜스포메이션 실행할 태스크 수에 직접적인 영향을 줌
  - 파티션 개수가 너무 적으면 -> 많은 데이터에서 아주 일부의 CPU/코어만 사용함 -> 성능 저하, 클러스터를 제대로 활용 못함
  - 파티션 개수가 너무 많다면 -> 실제 필요한 것보다 많은 자원을 사용하게 됨 -> 멀티테넌트 환경에서는 자원 부족 현상 발생
- Partitioner에 의해 RDD 파티셔닝이 실행된 파티셔너는 파티션 인덱스를 RDD 엘리먼트에 할당함

In [55]:
var_rdd_one = sc.parallelize((1,2,3))
var_rdd_one

Out[37]: ParallelCollectionRDD[215] at readRDDFromFile at PythonRDD.scala:332

In [56]:
var_rdd_one.getNumPartitions()

Out[39]: 8

## Shuffling
- 파티셔너가 어떤 파티션을 사용하든 많은 연산이 RDD의 파티션 전체에 걸쳐 데이터 리파티셔닝이 발생함
  - 새로운 타티션이 생성되거나 파티션이 축소, 병합될 수 있음
- 리파티셔닝에 필요한 모든 데이터 이동을 셔플링 shuffling이라고 함
  - 셔플링을 할 때, Disk I/O, Network I/O가 과도하게 발생함
  - 셔플링은 계산을 동일 익스큐터의 메모리에서 더 이상 진행하지 않고 익스큐터 간에 데이터를 교환함 -> 많은 성능 지연을 초래할 수 있음
  - 스파크 잡의 실행 프로세스를 결정, 잡이 스테이지로 분할되는 부분에 영향을 미침
  - 셔플링이 많을수록 스파크 잡이 실행될 때 더 많은 스테이지가 발생하기 때문에 성능에 영향을 미침
- 리파티셔닝을 유발하는 연산은 **조인**, 리듀스, 그룹핑, 집계 연산이 있음

### 버켓팅 Bucketing
- Bucketing = pre-(shuffle + sort) inputs on join Keys
- 각 파일에 저장된 데이터를 제어할 수 있는 또다른 파일 조직화 기법
- 동일한 버킷 ID를 가진 데이터가 하나의 물리적 파티션에 모두 모여있기 때문에 데이터를 읽을 때 셔플을 피할 수 있음
- 같은 키로 계속 조인이 발생하는 경우, 일별 누적으로 쌓여가는 테이블을 버킷팅을 하면 효과를 볼 수 있음

d ### Broadcast variable
- 브로드캐스트 변수는 모든 익스큐터에서 사용할 수 있는 공유 변수
- 드라이버에서 한 번 생성되면 익스큐터에서만 읽을 수 있음
- 전체 데이터셋이 스파크 클러스터에서 전파될 수 있어서 익스큐터에서는 브로드캐스ㅡ 변수의 데이터에 접근할 수 있음
- **익스큐터 내부에서 실행되는 모든 태스크는 모두 브로드캐스 변수에 접근할 수 있음**